In [1]:
# !pip install requests beautifulsoup4 pandas
# !pip install selenium
# !pip install lxml_html_clean
# !pip install requests-html
#!pip install scrapy

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import csv
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
#from requests_html import HTMLSession
import asyncio
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from requests_html import AsyncHTMLSession
import requests
from bs4 import BeautifulSoup
import re
import time
import random

In [3]:
BASE_URL = "https://steamdb.info"
SEARCH_URL = "https://steamdb.info/instantsearch/?page={page_id}"

num_games = 2000

In [4]:
def wait_for_page_load(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') in ['complete']
    )

In [5]:
def get_game_urls():
    game_urls = []

    print("LOADED")

    for page_index in range(1, 61):
        options = webdriver.EdgeOptions()
        options.page_load_strategy = 'normal'
        driver = webdriver.Edge(options=options)
        driver.get(SEARCH_URL.format(page_id = page_index))
        time.sleep(10)
        #wait_for_page_load(driver)
        try:
            hits_list = driver.find_element(By.CLASS_NAME, 'ais-Hits-list')
            games = hits_list.find_elements(By.CLASS_NAME, 'ais-Hits-item')
        except:
            page_index -= 1
            continue
        for game in games:
            link = game.find_element(By.TAG_NAME, 'a')
            if link:
                game_urls.append(link.get_attribute('href'))
        driver.quit()
    return game_urls

In [6]:
def load_game_urls_from_file(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file.readlines()]


In [7]:
def app_info_columns(driver):
    button = driver.find_element(By.XPATH, '//*[@id="tab-info"]')
    button.click()

def price_history_columns(driver):
    button = driver.find_element(By.XPATH, '//*[@id="tab-prices"]')
    button.click()

def charts_columns(driver):
    button = driver.find_element(By.XPATH, '//*[@id="tab-charts"]')
    button.click()

In [8]:
def scrape_game_info(url):
    options = webdriver.EdgeOptions()
    options.page_load_strategy = 'eager'
    driver = webdriver.Edge(options=options)
    print(url)
    driver.get(url)

    #wait_for_page_load(driver)

    game_info = {
        'NAME': 'N/A',                      # DONE
        'STORE_GENRE': 'N/A',               #app info
        'RATING_SCORE': 'N/A',          #app info
        'N_SUPPORTED_LANGUAGES': 'N/A',     #app info
        'DEVELOPERS': 'N/A',                # DONE
        'SUPPORTED_PLATFORMS': 'N/A',       # DONE
        'POSITIVE_REVIEWS': 'N/A',          # DONE
        'NEGATIVE_REVIEWS': 'N/A',          # DONE
        'TECHNOLOGIES': 'N/A',              # DONE
        'RELEASE_DATE': 'N/A',              # DONE
        'TOTAL_TWITCH_PEAK': 'N/A',         #charts
        'PRICE': 'N/A',                     #price tab
        'N_DLC': 'N/A',                     #main window
        '24_HOUR_PEAK': 'N/A'               #charts
    }


# NAME
    try:
        game_name_element = driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
        game_info['NAME'] = game_name_element.text
    except Exception as e:
        game_info['NAME'] = 'N/A'



# DEVELOPERS
    try:
        developer_element = driver.find_element(By.XPATH, '//td[text()="Developer"]/following-sibling::td/a')
        game_info['DEVELOPERS'] = developer_element.get_attribute("content").strip()  ######## چک شه اگر ارایه ای بود و جند نفر بودن هم درست کار کنه!!!
    except Exception as e:
        game_info['DEVELOPERS'] = 'N/A'



# SUPPORTED_PLATFORMS
    try:
        supported_systems_element = driver.find_element(By.CSS_SELECTOR, 'td.os-icons')
        game_info['SUPPORTED_PLATFORMS'] = supported_systems_element.text.strip()
    except Exception as e:
        game_info['SUPPORTED_PLATFORMS'] = 'N/A'

# POSITIVE_REVIEWS
    try:
        positive_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span.header-thing-good')
        game_info['POSITIVE_REVIEWS'] = re.sub(r'[^\d]', '', positive_reviews_element.text)
    except Exception as e:
        game_info['POSITIVE_REVIEWS'] = 'N/A'

# NEGATIVE_REVIEWS
    try:
        negative_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span.header-thing-poor')
        game_info['NEGATIVE_REVIEWS'] = re.sub(r'[^\d]', '', negative_reviews_element.text)
    except Exception as e:
        game_info['NEGATIVE_REVIEWS'] = 'N/A'
# TECHNOLOGIES
    try:
        technology_elements = driver.find_elements(By.XPATH, '//td[a[contains(@href, "/tech/")]]')
        technologies = [elem.text.strip() for elem in technology_elements if elem.text.strip() != '']
        game_info['TECHNOLOGIES'] = ", ".join(technologies)
    except Exception as e:
        game_info['TECHNOLOGIES'] = 'N/A'
# RELEASE_DATE
    try:
        release_date_label_element = driver.find_element(By.XPATH, '//td[text()="Release Date"]')
        release_date_element = release_date_label_element.find_element(By.XPATH, 'following-sibling::td')
        game_info['RELEASE_DATE'] = release_date_element.text
    except Exception as e:
        game_info['RELEASE_DATE'] = 'N/A'

    time.sleep(1)
## app info part
    app_info_columns(driver)
    tbody = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[2]/div[3]/table[1]/tbody')
    tr_tags = tbody.find_elements(By.TAG_NAME, 'tr')
    names_to_check = {'review_score': 'RATING_SCORE', 'Store Genres': 'STORE_GENRE'}
    for tr_tag in tr_tags:
        try:
            td_span3 = tr_tag.find_element(By.CLASS_NAME, 'span3')
            td_text = td_span3.text
            if td_text in names_to_check.keys():
                column_name = names_to_check[td_text]
                game_info[column_name] = tr_tag.text
        except Exception as e:
            print(f"An error occurred: {e}")




# N_SUPPORTED_LANGUAGES
    try:
        game_name_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[2]/div[3]/table[2]/thead/tr/th[1]')
        game_info['N_SUPPORTED_LANGUAGES'] = game_name_element.text
    except Exception as e:
        game_info['N_SUPPORTED_LANGUAGES'] = 'N/A'



## price history
    time.sleep(1)
    price_history_columns(driver)
# PRICE

    try:
        game_name_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[2]')
        game_info['PRICE'] = game_name_element.text
    except Exception as e:
        game_info['PRICE'] = 'N/A'

# N_DLC
    try:
        a_tag = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[1]/nav/a[5]')
        aria_controls_value = a_tag.get_attribute('aria-controls')
        if aria_controls_value == 'dlc':
            game_name_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[1]/nav/a[5]/span')
            game_info['N_DLC'] = game_name_element.text
        else:
            game_info['N_DLC'] = 'N/A'
    except Exception as e:
        game_info['N_DLC'] = 'N/A'

    time.sleep(1)
## charts
    charts_columns(driver)
# 24_HOUR_PEAK
    try:
        game_name_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[2]/div[4]/ul[1]/li[3]')
        game_info['24_HOUR_PEAK'] = game_name_element.text
    except Exception as e:
        game_info['24_HOUR_PEAK'] = 'N/A'
# TOTAL_TWITCH_PEAK
    try:
        game_name_element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div/div[2]/div/div[2]/div[4]/ul[1]/li[4]')
        game_info['TOTAL_TWITCH_PEAK'] = game_name_element.text
    except Exception as e:
        game_info['TOTAL_TWITCH_PEAK'] = 'N/A'




    print( game_info)
    print("_______________________________________________")

    time.sleep(1)
    driver.quit()
    return game_info


In [9]:
def write_to_csv(all_games_info):
    with open('games_info.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['NAME', 'STORE_GENRE', 'RATING_SCORE', 'N_SUPPORTED_LANGUAGES', 'DEVELOPERS', 'SUPPORTED_PLATFORMS', 'POSITIVE_REVIEWS', 'NEGATIVE_REVIEWS', 'TECHNOLOGIES', 'RELEASE_DATE', 'TOTAL_TWITCH_PEAK', 'PRICE', 'N_DLC', '24_HOUR_PEAK']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for game in all_games_info:
            writer.writerow(game)

    print(f"Collected information for {len(all_games_info)} games.")

In [10]:
import pandas as pd
def load_game_info_from_file(filename):
    df = pd.read_csv(filename)
    list_of_dicts = df.to_dict(orient='records')
    return list_of_dicts

In [ ]:
checkpoint_interval = 5

def main():
    game_urls = []
    if os.path.exists('game_urls.txt'):
        game_urls = load_game_urls_from_file('game_urls.txt')
    else:
        game_urls = get_game_urls()
        with open('game_urls.txt', 'w') as f:
            for url in game_urls:
                f.write(f"{url}\n")

    print(f"Collected {len(game_urls)} game URLs.")


    if os.path.exists('games_info.csv'):
        all_games_info = load_game_info_from_file('games_info.csv')
    else:
        all_games_info = []


    url_index = 0
    for url in game_urls:
        url_index += 1
        print(url_index)
        print(len(game_urls))
        url = game_urls.pop()
        try:
            game_info = scrape_game_info(url)
            if game_info == None:
                game_urls.append(url)
                url_index -= 1
                random.shuffle(game_urls)
            else :
                all_games_info.append(game_info)
            time.sleep(2)

        except:
            game_urls.append(url)
            random.shuffle(game_urls)
            url_index -= 1
            time.sleep(5)
            continue

        if url_index % checkpoint_interval == 0:
            write_to_csv(all_games_info)
            with open('game_urls.txt', 'w') as f:
                for url_1 in game_urls:
                    f.write(f"{url_1}\n")

if __name__ == "__main__":
    main()


Collected 484 game URLs.
1
484

1
484
https://steamdb.info/app/1130620/
{'NAME': 'Aka Manto | 赤マント', 'STORE_GENRE': 'Store Genres Action (1), Adventure (25), Casual (4), Indie (23), Simulation (28), Strategy (2)', 'RATING_SCORE': 'review_score 8', 'N_SUPPORTED_LANGUAGES': '3 Languages', 'DEVELOPERS': "Chilla's Art", 'SUPPORTED_PLATFORMS': 'N/A', 'POSITIVE_REVIEWS': '812', 'NEGATIVE_REVIEWS': '101', 'TECHNOLOGIES': 'Engine.Unity', 'RELEASE_DATE': '24 September 2019 – 04:11:09 UTC (5 years ago)', 'TOTAL_TWITCH_PEAK': '19\nall-time peak 3 years ago15 February 2021', 'PRICE': '$4.99', 'N_DLC': 'N/A', '24_HOUR_PEAK': '8\n24-hour peak'}
_______________________________________________
2
483
https://steamdb.info/app/2397840/
{'NAME': 'Owinka Shooter 2', 'STORE_GENRE': 'Store Genres Action (1), Adventure (25), Indie (23)', 'RATING_SCORE': 'review_score 7', 'N_SUPPORTED_LANGUAGES': '1 Languages', 'DEVELOPERS': 'Owinka', 'SUPPORTED_PLATFORMS': 'Windows', 'POSITIVE_REVIEWS': '36', 'NEGATIVE_REVIEW